In [1]:
!pip install "autogen-agentchat[retrievechat-qdrant]~=0.2" "flaml[automl]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of grpcio-tools to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of grpcio-tools to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of opentelemetry-proto to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of opentelemetry-sdk to determine which v

In [6]:
import json


from google.colab import userdata
api_key=userdata.get('open_ai')
config=[
    {
        "model": "gpt-4-turbo-preview",
        "api_key":api_key
    }
]
with open("OAI_CONFIG_LIST", "w") as f:
    json.dump(config, f)

config_list = autogen.config_list_from_json("OAI_CONFIG_LIST")

In [3]:
from qdrant_client import QdrantClient
from sentence_transformers import SentenceTransformer

import autogen
from autogen import AssistantAgent
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent

# Accepted file formats for that can be stored in
# a vector database instance
from autogen.retrieve_utils import TEXT_FORMATS

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [4]:
print("Accepted file formats for `docs_path`:")
print(TEXT_FORMATS)

Accepted file formats for `docs_path`:
['txt', 'json', 'csv', 'tsv', 'md', 'html', 'htm', 'rtf', 'rst', 'jsonl', 'log', 'xml', 'yaml', 'yml', 'pdf']


In [7]:
# 1. create an AssistantAgent instance named "assistant"
assistant = AssistantAgent(
    name="assistant",
    system_message="You are a helpful assistant.",
    llm_config={
        "timeout": 600,
        "cache_seed": 42,
        "config_list": config_list,
    },
)

# Optionally create embedding function object
sentence_transformer_ef = SentenceTransformer("all-distilroberta-v1").encode
client = QdrantClient(":memory:")

# 2. create the RetrieveUserProxyAgent instance named "ragproxyagent"
# Refer to https://microsoft.github.io/autogen/docs/reference/agentchat/contrib/retrieve_user_proxy_agent
# and https://microsoft.github.io/autogen/docs/reference/agentchat/contrib/vectordb/qdrant
# for more information on the RetrieveUserProxyAgent and QdrantVectorDB
ragproxyagent = RetrieveUserProxyAgent(
    name="ragproxyagent",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    retrieve_config={
        "task": "code",
        "docs_path": [
            "https://raw.githubusercontent.com/microsoft/flaml/main/README.md",
            "https://raw.githubusercontent.com/microsoft/FLAML/main/website/docs/Research.md",
        ],  # change this to your own path, such as https://raw.githubusercontent.com/microsoft/autogen/main/README.md
        "chunk_token_size": 2000,
        "model": config_list[0]["model"],
        "db_config": {"client": client},
        "vector_db": "qdrant",  # qdrant database
        "get_or_create": True,  # set to False if you don't want to reuse an existing collection
        "overwrite": True,  # set to True if you want to overwrite an existing collection
        "embedding_function": sentence_transformer_ef,  # If left out fastembed "BAAI/bge-small-en-v1.5" will be used
    },
    code_execution_config=False,
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.3k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/653 [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
# reset the assistant. Always reset the assistant before starting a new conversation.
assistant.reset()

qa_problem = "Is there a function called tune_automl?"
chat_results = ragproxyagent.initiate_chat(assistant, message=ragproxyagent.message_generator, problem=qa_problem)

Trying to create collection.


2024-12-27 19:04:50,107 - autogen.agentchat.contrib.retrieve_user_proxy_agent - INFO - Found 3 chunks.


VectorDB returns doc_ids:  [['0d580ff7-7fc1-bf18-cd0c-de037cbd9343', '0ecd7192-3761-7d6f-9151-5ff504ca740b', '9be3ff7b-0243-32ab-5e97-7762580e2663']]
Adding content of doc 0d580ff7-7fc1-bf18-cd0c-de037cbd9343 to context.
Adding content of doc 0ecd7192-3761-7d6f-9151-5ff504ca740b to context.
Adding content of doc 9be3ff7b-0243-32ab-5e97-7762580e2663 to context.
ragproxyagent (to assistant):

You're a retrieve augmented coding assistant. You answer user's questions based on your own knowledge and the
context provided by the user.
If you can't answer the question with or without the current context, you should reply exactly `UPDATE CONTEXT`.
For code generation, you must obey the following rules:
Rule 1. You MUST NOT install any packages because all the packages needed are already installed.
Rule 2. You must follow the formats below to write your code:
```language
# your code
```

User's question is: Is there a function called tune_automl?

Context is: [![PyPI version](https://badge.fury.

In [9]:
# reset the assistant. Always reset the assistant before starting a new conversation.
assistant.reset()

qa_problem = "Who is the author of FLAML?"
chat_results = ragproxyagent.initiate_chat(assistant, message=ragproxyagent.message_generator, problem=qa_problem)

VectorDB returns doc_ids:  [['0d580ff7-7fc1-bf18-cd0c-de037cbd9343', '0ecd7192-3761-7d6f-9151-5ff504ca740b', '9be3ff7b-0243-32ab-5e97-7762580e2663']]
Adding content of doc 0d580ff7-7fc1-bf18-cd0c-de037cbd9343 to context.
Adding content of doc 0ecd7192-3761-7d6f-9151-5ff504ca740b to context.
Adding content of doc 9be3ff7b-0243-32ab-5e97-7762580e2663 to context.
ragproxyagent (to assistant):

You're a retrieve augmented coding assistant. You answer user's questions based on your own knowledge and the
context provided by the user.
If you can't answer the question with or without the current context, you should reply exactly `UPDATE CONTEXT`.
For code generation, you must obey the following rules:
Rule 1. You MUST NOT install any packages because all the packages needed are already installed.
Rule 2. You must follow the formats below to write your code:
```language
# your code
```

User's question is: Who is the author of FLAML?

Context is: [![PyPI version](https://badge.fury.io/py/FLAML.

In [14]:
type(chat_results)

autogen.agentchat.chat.ChatResult

In [21]:
chat_results.chat_history[-1]['content']

'The authors of FLAML, as mentioned in the provided research publications, include Chi Wang, Qingyun Wu, Markus Weimer, Erkang Zhu, Silu Huang, Amin Saied, Susan Xueqing Liu, John Langford, Paul Mineiro, Marco Rossi, and collaborators from institutions such as Penn State University, Stevens Institute of Technology, University of Washington, and University of Waterloo.'

In [22]:
# reset the assistant. Always reset the assistant before starting a new conversation.
assistant.reset()

# set `human_input_mode` to be `ALWAYS`, so the agent will ask for human input at every step.
ragproxyagent.human_input_mode = "ALWAYS"
code_problem = "how to build a time series forecasting model for stock price using FLAML?"
chat_result = ragproxyagent.initiate_chat(assistant, message=ragproxyagent.message_generator, problem=code_problem)

VectorDB returns doc_ids:  [['0d580ff7-7fc1-bf18-cd0c-de037cbd9343', '0ecd7192-3761-7d6f-9151-5ff504ca740b', '9be3ff7b-0243-32ab-5e97-7762580e2663']]
Adding content of doc 0d580ff7-7fc1-bf18-cd0c-de037cbd9343 to context.
Adding content of doc 0ecd7192-3761-7d6f-9151-5ff504ca740b to context.
Adding content of doc 9be3ff7b-0243-32ab-5e97-7762580e2663 to context.
ragproxyagent (to assistant):

You're a retrieve augmented coding assistant. You answer user's questions based on your own knowledge and the
context provided by the user.
If you can't answer the question with or without the current context, you should reply exactly `UPDATE CONTEXT`.
For code generation, you must obey the following rules:
Rule 1. You MUST NOT install any packages because all the packages needed are already installed.
Rule 2. You must follow the formats below to write your code:
```language
# your code
```

User's question is: how to build a time series forecasting model for stock price using FLAML?

Context is: [!

In [27]:
last_content = chat_results.chat_history[-1]['content']
import markdown
from markdownify import markdownify
# Convert to Markdown
markdown_content = markdown.markdown(last_content)

markdown_content = markdownify(markdown_content)

# Print the Markdown content
print(markdown_content)

print(markdown_content)



```python
from flaml import AutoML
from sklearn.model\_selection import train\_test\_split
import pandas as pd

Assuming `df` is your DataFrame containing the stock price data with a 'Date' column and a 'Close' column

First, let's convert 'Date' to datetime type if it's not already

df['Date'] = pd.to\_datetime(df['Date'])

Setting 'Date' as the index

df.set\_index('Date', inplace=True)

Let's say 'Close' is the column we want to predict

For a time series forecasting model, we should split the data based on time

split\_date = '2023-01-01' # This is an example split date, adjust based on your dataset

Splitting the data into train and test sets

train = df.loc[df.index < split\_date]
test = df.loc[df.index >= split\_date]

For time series forecasting, it's common to use previous values as features

Here, you can generate features based on your requirement, like lags of 'Close'

For simplicity, we'll use only 'Close' as our feature to train the model

X\_train, y\_train = train[['C

In [28]:
code_problem = "How can I use FLAML to perform a classification task and use spark to do parallel training. Train 30 seconds and force cancel jobs if time limit is reached."
chat_result = ragproxyagent.initiate_chat(
    assistant, message=ragproxyagent.message_generator, problem=code_problem, search_string="spark"
)  # search_st

VectorDB returns doc_ids:  [['0d580ff7-7fc1-bf18-cd0c-de037cbd9343', '0ecd7192-3761-7d6f-9151-5ff504ca740b', '9be3ff7b-0243-32ab-5e97-7762580e2663']]
Adding content of doc 0d580ff7-7fc1-bf18-cd0c-de037cbd9343 to context.
Adding content of doc 0ecd7192-3761-7d6f-9151-5ff504ca740b to context.
Adding content of doc 9be3ff7b-0243-32ab-5e97-7762580e2663 to context.
ragproxyagent (to assistant):

You're a retrieve augmented coding assistant. You answer user's questions based on your own knowledge and the
context provided by the user.
If you can't answer the question with or without the current context, you should reply exactly `UPDATE CONTEXT`.
For code generation, you must obey the following rules:
Rule 1. You MUST NOT install any packages because all the packages needed are already installed.
Rule 2. You must follow the formats below to write your code:
```language
# your code
```

User's question is: How can I use FLAML to perform a classification task and use spark to do parallel trainin

In [39]:
chat_result.chat_history[-3]['content']

'Using FLAML for classification with parallel training via Spark and enforcing a time limit for the jobs requires combining FLAML\'s AutoML capabilities with Spark\'s distributed computing framework. However, FLAML does not natively support distributed training across Spark clusters directly as of the latest information available. To perform parallel training, you\'d typically need to handle the distribution aspect separately, perhaps by distributing the cross-validation folds or training different models on different Spark executors. \n\nHere\'s a conceptual approach to achieve your objective, assuming you have a Spark environment set up and FLAML installed. This code does not directly run tasks in parallel across Spark but outlines how you might structure your code for a similar effect:\n\n1. **Setup Spark Session**: Ensure you have SparkSession created and configured according to your cluster setup.\n2. **Define FLAML Training Function**: This function will use FLAML to train a mode